In [19]:
import requests
import xml.etree.ElementTree as ET
import os
import re
import pickle
from dotenv import load_dotenv
import getpass
from datetime import date

# Where to store session:
SESSION_FILE = ".balanz_session.pkl"

def save_session(access_token, cookie, id_cuenta):
    with open(SESSION_FILE, "wb") as f:
        pickle.dump((access_token, cookie, id_cuenta), f)

def load_session():
    if not os.path.exists(SESSION_FILE):
        return None, None, None
    try:
        with open(SESSION_FILE, "rb") as f:
            return pickle.load(f)
    except:
        return None, None, None

def login(username, password):
    session = requests.Session()
    # ---------- 1. Get NONCE ----------
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Origin": "https://clientes.balanz.com",
        "Referer": "https://clientes.balanz.com/auth/login?avoidAuthRedirect=true",
        "Content-Type": "application/json"
    }
    resp = session.post(
        "https://clientes.balanz.com/api/v1/auth/init?avoidAuthRedirect=true",
        json={"user": username}, headers=headers
    )
    print("DEBUG: Nonce status:", resp.status_code)
    print("DEBUG: Nonce headers:", resp.headers)
    print("DEBUG: Nonce response:", resp.text)

    # Parse the nonce value from the XML
    root = ET.fromstring(resp.text)
    nonce = root.findtext("nonce")
    print("DEBUG: Extracted nonce:", nonce)
    if not nonce:
        raise Exception("Could not get nonce")

    # Extract cookiesession1 from SET-COOKIE NOW (not after login)
    raw_cookie = resp.headers.get("Set-Cookie", "")
    m = re.search(r'cookiesession1=([^;]+)', raw_cookie)
    cookiesession1 = m.group(1) if m else None
    print("DEBUG: cookiesession1 after INIT:", cookiesession1)
    if not cookiesession1:
        raise Exception("No cookiesession1 set after init! Something is wrong.")

    # ---------- 2. LOGIN ----------
    login_xml = f"""<jsonObject><user>{username}</user><pass>{password}</pass><nonce>{nonce}</nonce></jsonObject>"""
    headers["Content-Type"] = "application/xml"
    resp2 = session.post(
        "https://clientes.balanz.com/api/v1/auth/login?avoidAuthRedirect=true",
        data=login_xml, headers=headers
    )
    print("\nDEBUG: LOGIN status:", resp2.status_code)
    print("DEBUG: LOGIN headers:", resp2.headers)
    print("DEBUG: LOGIN response (raw):", resp2.text)

    if "Sesiones activas" in resp2.text:
        raise Exception("Too many active sessions. Wait and try again.")
    if "idError" in resp2.text:
        # Optional: extract and print error description
        try:
            root2 = ET.fromstring(resp2.text)
            mensaje = root2.findtext("Descripcion")
            print(f"DEBUG: Login error description: {mensaje}")
        except Exception:
            print("DEBUG: Failed to parse login error XML")
        raise Exception("Login failed.")

    try:
        root2 = ET.fromstring(resp2.text)
    except Exception as e:
        print("DEBUG: Could not parse login XML! Exception:", e)
        print("DEBUG: LOGIN RESPONSE:", resp2.text)
        raise

    access_token = root2.findtext("AccessToken")
    id_cuenta = root2.findtext("idPersona")  # Or adjust if API returns a different field
    print(f"DEBUG: Extracted AccessToken: {access_token}")
    print(f"DEBUG: Extracted id_cuenta/idPersona: {id_cuenta}")

    # Do NOT try to parse Set-Cookie from LOGIN (resp2), only use value from INIT (resp)
    print(f"DEBUG: Login complete. Using AccessToken={access_token}, id_cuenta={id_cuenta}, session cookie={cookiesession1}")

    if not (access_token and cookiesession1 and id_cuenta):
        print("DEBUG: One or more required values missing!")
        raise Exception("Login failed, no access/cookie/id")

    # Optionally return session *object* if you want to use it for requests with cookies attached
    return access_token, cookiesession1, id_cuenta  # Optionally: , session

def session_test_call(access_token, cookie, id_cuenta):
    # Try a lightweight authenticated call; here we use the "cotizacioninstrumento" endpoint as an example.
    url = "https://clientes.balanz.com/api/v1/cotizacioninstrumento"
    headers = {
        "Authorization": access_token,
        "Cookie": f"cookiesession1={cookie}",
        "Accept": "application/json",
        "User-Agent": "Mozilla/5.0"
    }
    params = {
        "plazo": "1",
        "idCuenta": id_cuenta,
        "ticker": "S2S5C"  # Put a valid, but simple/cheap ticker, e.g. known to always work
    }
    try:
        resp = requests.get(url, headers=headers, params=params, timeout=6)
        # If forbidden/expired
        if resp.status_code == 403 and "Sesion Expirada" in resp.text:
            return False
        if resp.status_code == 401:
            return False
        # If error codes in json
        try:
            data = resp.json()
            if "Sesion Expirada" in data.get("Descripcion",""):
                return False
        except Exception:
            pass
        # Otherwise session OK
        return True
    except:
        return False

def get_balanz_session(username, password):
    access_token, cookie, id_cuenta = load_session()
    print(f"DEBUG: Loaded session: AccessToken={access_token}, cookie={cookie}, id_cuenta={id_cuenta}") # DEBUG
    if access_token and cookie and id_cuenta:
        ok = session_test_call(access_token, cookie, id_cuenta)
        print(f"DEBUG: session_test_call returned: {ok}") # DEBUG
        if ok:
            print("Using previous session.")
            return access_token, cookie, id_cuenta
        else:
            print("Session expired or invalid, logging in again...")
    else:
        print("No stored session, logging in...")

    access_token, cookie, id_cuenta = login(username, password)
    print("New session obtained.")
    save_session(access_token, cookie, id_cuenta)
    return access_token, cookie, id_cuenta

In [20]:
# Load .env variables into environment
load_dotenv()

# Read credentials
username = os.getenv("BALANZ_USER")
password = os.getenv("BALANZ_PASSWORD")

if not username:
    username = input("Please enter Balanz username: ")

if not password:
    password = getpass.getpass("Please enter Balanz password: ")

access_token, cookie, id_cuenta = get_balanz_session(username, password)
# You are now logged in and ready to use these credentials!

# Use access_token, cookie, id_cuenta for further API calls:
url = "https://clientes.balanz.com/api/v1/cotizacioninstrumento"
headers = {
    "Authorization": access_token,
    "Cookie": f"cookiesession1={cookie}",
    "Accept": "application/json",
    "User-Agent": "Mozilla/5.0"
}
params = {
    "plazo": "2",
   # "idCuenta": id_cuenta,
    "ticker": "S2S5C"
}
resp = requests.get(url, headers=headers, params=params)
print(resp.status_code, resp.text)

DEBUG: Loaded session: AccessToken=C61DB0AA-E768-401F-BE13-85A85E5B87DA, cookie=678ADAB8AD4B2495B22BD11A75DB62FC, id_cuenta=1205083
DEBUG: session_test_call returned: True
Using previous session.
200 {"Cotizacion":{"Source":"BZMD","Identifier":"S2S5C-0002-C-CT-EXT","NivelDiferido":0,"SecurityID":"S2S5C-0002-C-CT-EXT","Descripcion":"LETRAS DEL TESORO CAP $ V30/09/25","idPlazo":1,"plazo":"24hs","idTipo":15,"tipo":"Letras","panel":15,"TickerHistorico":"S2S5C","ticker":"S2S5C","plazoHistorico":1,"SimboloMoneda":"usd","idMoneda":4,"UltimoPrecio":0.00105,"Variacion":"-","pcv":-0.00004,"pcp":-3.67,"MejorOfertaCompra":"-","MejorOfertaVenta":"-","UltimaOperacion":"2025-09-16 13:04:33.0","PrecioMaximo":0.00106,"PrecioMinimo":0.00105,"PrecioCierreAnterior":0.00105,"PrecioApertura":0.00106,"Volumen":91872,"idWatchlist":0,"CantCompra":0,"PrecioCompra":0,"PrecioVenta":0,"CantVenta":0,"verProfundidad":1,"Contratos":1,"MargenRequerido":0,"ListaPlazos":"0,1","TextoAlerta":"","SecurityStatus":"Abierto",

In [21]:
url = "https://clientes.balanz.com/api/v1/graficar"
params = {
    "ticker": "S2S5C",
    "idPlazo": 1,
    "desde": "2024-01-01",
    "hasta": date.today().strftime("%Y-%m-%d")
}
headers = {
    "Authorization": access_token,
    "Cookie": f"cookiesession1={cookie}",
    "Accept": "application/json",
    "User-Agent": "Mozilla/5.0"
}
r = requests.get(url, headers=headers, params=params)
print(r.status_code)
print(r.json())

404


JSONDecodeError: Expecting value: line 1 column 1 (char 0)